First download cat-mel_2bar_big.tar from [https://github.com/magenta/magenta/tree/main/magenta/models/music_vae](https://github.com/magenta/magenta/tree/main/magenta/models/music_vae)

In [2]:
from magenta.models.music_vae import configs, TrainedModel
from note_seq.protobuf import music_pb2

2024-11-26 20:22:28.242937: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-11-26 20:22:28.243016: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/eiffert/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
checkpoint_dir_or_path = "./cat-mel_2bar_big.tar"
config = configs.CONFIG_MAP["cat-mel_2bar_big"]
temperature = 0.5

model = TrainedModel(config, batch_size=2, checkpoint_dir_or_path=checkpoint_dir_or_path)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 2, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

Instructions for updating:
Use `tf.cast` instead.


/home/eiffert/.local/lib/python3.8/site-packages/magenta/models/music_vae/lstm_utils.py:94: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tf.layers.dense(
/home/eiffert/.local/lib/python3.8/site-packages/magenta/contrib/rnn.py:749: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._kernel = self.add_variable(
/home/eiffert/.local/lib/python3.8/site-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._bias = self.add_variable(


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


2024-11-26 20:22:42.574866: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-11-26 20:22:42.574962: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-11-26 20:22:42.575046: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (N-EIFFERT): /proc/driver/nvidia/version does not exist
/home/eiffert/.local/lib/python3.8/site-packages/magenta/models/music_vae/base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.
INFO:tensorflow:Unbundling checkpoint.


/home/eiffert/.local/lib/python3.8/site-packages/magenta/models/music_vae/base_model.py:200: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  sigma = tf.layers.dense(
2024-11-26 20:22:42.859870: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Restoring parameters from /tmp/tmpcck3ng7r/cat-mel_2bar_big.ckpt


2024-11-26 20:24:48.967367: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


In [4]:
def contains_c(sequence: music_pb2.NoteSequence) -> int:
	found = False
	for note in sequence.notes:
		if (note.pitch % 12) == 0:
			found = True
	return 0 if found else 30

def pentatonic_dist_notes(sequence: music_pb2.NoteSequence) -> int:
    distance = 0
    dists = [0, 30, 0, 30, 0, 1, 30, 0, 30, 0, 30, 1]
    for note in sequence.notes:
        pitch = note.pitch
        distance += dists[pitch % 12]
    return distance

def range(sequence: music_pb2.NoteSequence) -> int:
    low = 127
    high = 0
    for note in sequence.notes:
        pitch = note.pitch
        low = min(low, pitch)
        high = max(high, pitch)
    return high-low

def number_of_notes(sequence: music_pb2.NoteSequence) -> int:
    return len(sequence.notes)

def penalty(sequence: music_pb2.NoteSequence) -> int:
    score = contains_c(sequence) \
            + pentatonic_dist_notes(sequence) \
            + max(0, 4 - range(sequence)) * 5 \
            + max(0, range(sequence) - 12) * 10 \
            + max(0, 5 - number_of_notes(sequence)) * 20 \
            + max(0, number_of_notes(sequence) - 16) * 10
    return score

results = model.sample(
        n=1024,
        length=config.hparams.max_seq_len,
        temperature=temperature)

pentatonic_dist small

contains the note C

range at least a third (4 semitones)

range not more than an octave (12 semitones)

at least 5 notes

less than 16 notes

In [5]:
scores = [penalty(result) for result in results]

good_results = [result for result in results if penalty(result) < 5]

print(len(good_results))

40


In [6]:
import os
import time

from magenta.models.melody_rnn import melody_rnn_model
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator
from magenta.models.shared import sequence_generator_bundle
import note_seq
from note_seq.protobuf import generator_pb2
from note_seq.protobuf import music_pb2

config_id = "lookback_rnn"
melody_rnn_config = melody_rnn_model.default_configs[config_id]
bundle_file = config_id+".mag"
bundle = sequence_generator_bundle.read_bundle_file(bundle_file)

generator = melody_rnn_sequence_generator.MelodyRnnSequenceGenerator(
      model=melody_rnn_model.MelodyRnnModel(melody_rnn_config),
      details=melody_rnn_config.details,
      steps_per_quarter=melody_rnn_config.steps_per_quarter,
      checkpoint=None,
      bundle=bundle)

In [7]:
def generate_song_section(our_sequence: music_pb2.NoteSequence, temperature: float = 0.70, num_steps: int = 128):
    primer_sequence = None
    qpm = note_seq.DEFAULT_QUARTERS_PER_MINUTE
    primer_sequence = our_sequence

    # Derive the total number of seconds to generate based on the QPM of the
    # priming sequence and the num_steps flag.
    seconds_per_step = 60.0 / qpm / generator.steps_per_quarter
    total_seconds = num_steps * seconds_per_step

    # Specify start/stop time for generation based on starting generation at the
    # end of the priming sequence and continuing until the sequence is num_steps
    # long.
    generator_options = generator_pb2.GeneratorOptions()

    input_sequence = primer_sequence
        # Set the start time to begin on the next step after the last note ends.

    last_end_time = max(n.end_time for n in primer_sequence.notes)

    generator_options.generate_sections.add(
        start_time=last_end_time + seconds_per_step,
        end_time=total_seconds)

    generator_options.args['temperature'].float_value = temperature
    generator_options.args['beam_size'].int_value = 1
    generator_options.args['branch_factor'].int_value = 1
    generator_options.args[
        'steps_per_iteration'].int_value = 1

    # Make the generate request num_outputs times and save the output as midi
    # files.
    generated_sequence = generator.generate(input_sequence, generator_options)

    return generated_sequence


no really long notes
similar number of distinct pitches
similar range
has some self-similarity
no notes shorter than stem

In [8]:
import os, csv, re, json, random

genres = {}
metadata = {}
with open("harmonixset/dataset/metadata.csv") as file:
    reader = csv.DictReader(file)
    for line in reader:
        genre = line["Genre"]
        genres[genre] = genres.get(genre, 0)+1
        metadata[line["File"]] = line

allowed_genres = set([
    "Pop",
    "Country",
    "Rock"
])

def is_allowed_file(basefile):
    return (metadata[basefile]["Genre"] in allowed_genres) \
        and ("Ultra Dance" not in metadata[basefile]["Release"])

def get_all_segments():
    all_segments = {}
    fieldnames = ["boundary_time_stamp", "label"]
    dirname = "harmonixset/dataset/segments"
    segment_filenames = os.listdir(dirname)
    for filename in segment_filenames:
        basefile = os.path.splitext(filename)[0]
        if not is_allowed_file(basefile):
            continue
        with open(os.path.join(dirname, filename)) as file:
            reader = csv.DictReader(file, delimiter=" ", fieldnames=fieldnames)
            segments = list(reader)
            good_segments = True
            for section in segments:
                if section["label"] == "section":
                    good_segments = False
            if good_segments:
                all_segments[basefile] = segments
    return all_segments


all_segments = get_all_segments()
types = set()
for file, segments in all_segments.items():
    for segment in segments:
        types.add(segment["label"])

merge_segments = {
    "altchorus": "chorus",
    "bigoutro": "outro",
    "bre": "instrumental",
    "break": "instrumental",
    "breakdown": "instrumental",
    "build": "instrumental",
    "chorus_instrumental": "instrumental",
    "chorushalf": "chorus",
    "chorusinst": "instrumental",
    "choruspart": "chorus",
    "drumroll": "instrumental",
    "fadein": "intro",
    "fast": "instrumental",
    "gtr": "instrumental",
    "gtrbreak": "instrumental",
    "guitar": "instrumental",
    "guitarsolo": "instrumental",
    "inst": "instrumental",
    "instbridge": "instrumental",
    "instchorus": "instrumental",
    "instintro": "intro",
    "instrumentalverse": "instrumental",
    "intchorus": "instrumental",
    "introchorus": "intro",
    "intropt": "intro",
    "introverse": "verse",
    "mainriff": "instrumental",
    "miniverse": "verse",
    "oddriff": "instrumental",
    "opening": "intro",
    "outroa": "outro",
    "postverse": "verse",
    "preverse": "verse",
    "quiet": "instrumental",
    "quietchorus": "chorus",
    "raps": "verse",
    "refrain": "chorus",
    "rhythmlessintro": "intro",
    "saxobeat": "instrumental",
    "section": "unknown",
    "silence": "instrumental",
    "slow": "instrumental",
    "slowverse": "verse",
    "solo": "instrumental",
    "stutter": "instrumental",
    "synth": "instrumental",
    "transition": "instrumental",
    "transitiona": "instrumental",
    "verse_slow": "verse",
    "versea": "verse",
    "verseinst": "instrumental",
    "versepart": "verse",
    "vocaloutro": "outro",
    "worstthingever": "instrumental"
}

def clean(label):
    label = re.sub(r'[0-9]+', '', label)
    label = merge_segments.get(label, label)
    return label

def get_clean_types():
    return list(set([clean(x) for x in types]))


def probs_from_prev2_num_chorus():
    probs = {}
    for file, segments in all_segments.items():
        chorus = 0
        since_chorus = 0
        prevprev = "none"
        prev = "start"
        for segment in segments:
            type = clean(segment["label"])

            twoprev = prevprev+","+prev
            twoprev = twoprev + " " + str(chorus) + "," + str(since_chorus)

            probs[twoprev] = probs.get(twoprev, {})
            probs[twoprev][type] = probs[twoprev].get(type, 0)+1

            
            if type == "chorus" and prev != "chorus":
                chorus += 1
                since_chorus = 0
            else:
                since_chorus += 1

            prevprev = prev
            prev = type

    # print(json.dumps(probs))
    return probs

def generate_that():
    probs = probs_from_prev2_num_chorus()

    chorus = 0
    since_chorus = 0
    prevprev = "none"
    prev = "start"
    output = []
    while prev != "end":
        twoprev = prevprev+","+prev
        twoprev = twoprev + " " + str(chorus) + "," + str(since_chorus)

        type = random.choices(list(probs[twoprev].keys()), weights=probs[twoprev].values())[0]

        if type == "chorus" and prev != "chorus":
            chorus += 1
            since_chorus = 0
        else:
            since_chorus += 1

        prevprev = prev
        prev = type

        if type != 'end':
            output.append(type)
    
    return output

In [9]:
generate_that()

['intro',
 'verse',
 'prechorus',
 'chorus',
 'verse',
 'chorus',
 'instrumental',
 'chorus']

In [10]:
from enum import Enum

class SectionType(Enum):
    INTRO = "intro"
    VERSE = "verse"
    CHORUS = "chorus"
    BRIDGE = "bridge"
    OUTRO = "outro"
    PRECHORUS = "prechorus"
    INSTRUMENTAL = "instrumental"
    POSTCHORUS = "postchorus"

from typing import List, Tuple, Dict

# gives song structure with VERSE, CHORUS, BRIDGE, PRECHORUS, POSTCHORUS, INTRO, OUTRO
def get_song_structure() -> List[SectionType]:
    return [SectionType[section.upper()] for section in generate_that()]

# can take VERSE, CHORUS, BRIDGE, PRECHORUS, POSTCHORUS, INTRO, OUTRO
def get_melodies(sections: List[SectionType]) -> Dict[SectionType, music_pb2.NoteSequence]:
    melodies = {}
    for i, section in enumerate(sections):
        melodies[section] = generate_song_section(good_results[i])
    return melodies



def get_song_melodies() -> List[Tuple[music_pb2.NoteSequence, SectionType]]:
    sections = get_song_structure()
    melodies = get_melodies(list(set(sections)))
    output = [(section, melodies[section]) for section in sections]
    return output

In [11]:
get_song_melodies()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmpfz8enfuo/model.ckpt


2024-11-26 20:33:10.288023: W tensorflow/core/framework/op_def_util.cc:371] Op TensorArray is deprecated. It will cease to work in GraphDef version 16. Use TensorArrayV3.
2024-11-26 20:33:10.288331: W tensorflow/core/framework/op_def_util.cc:371] Op TensorArrayScatter is deprecated. It will cease to work in GraphDef version 19. Use TensorArrayGradV3.
2024-11-26 20:33:10.288565: W tensorflow/core/framework/op_def_util.cc:371] Op TensorArrayRead is deprecated. It will cease to work in GraphDef version 16. Use TensorArrayReadV3.
2024-11-26 20:33:10.289503: W tensorflow/core/framework/op_def_util.cc:371] Op TensorArrayWrite is deprecated. It will cease to work in GraphDef version 16. Use TensorArrayWriteV3.
2024-11-26 20:33:10.289608: W tensorflow/core/framework/op_def_util.cc:371] Op TensorArraySize is deprecated. It will cease to work in GraphDef version 16. Use TensorArraySizeV3.
2024-11-26 20:33:10.289679: W tensorflow/core/framework/op_def_util.cc:371] Op TensorArrayGather is deprecat

INFO:tensorflow:Beam search yields sequence with log-likelihood: -111.628944 
INFO:tensorflow:Beam search yields sequence with log-likelihood: -68.661995 
INFO:tensorflow:Beam search yields sequence with log-likelihood: -101.161011 
INFO:tensorflow:Beam search yields sequence with log-likelihood: -50.536484 


[(<SectionType.INTRO: 'intro'>,
  ticks_per_quarter: 220
  tempos {
    qpm: 120.0
  }
  notes {
    pitch: 29
    velocity: 100
    end_time: 0.375
  }
  notes {
    pitch: 24
    velocity: 100
    start_time: 0.375
    end_time: 0.5
  }
  notes {
    pitch: 36
    velocity: 100
    start_time: 0.5
    end_time: 1.5
  }
  notes {
    pitch: 29
    velocity: 100
    start_time: 1.5
    end_time: 2.0
  }
  notes {
    pitch: 36
    velocity: 100
    start_time: 2.0
    end_time: 3.875
  }
  notes {
    pitch: 35
    velocity: 100
    start_time: 4.5
    end_time: 8.375
  }
  notes {
    pitch: 31
    velocity: 100
    start_time: 9.875
    end_time: 16.0
  }
  total_time: 16.0),
 (<SectionType.CHORUS: 'chorus'>,
  ticks_per_quarter: 220
  tempos {
    qpm: 120.0
  }
  notes {
    pitch: 41
    velocity: 100
    end_time: 0.375
  }
  notes {
    pitch: 40
    velocity: 100
    start_time: 0.375
    end_time: 0.5
  }
  notes {
    pitch: 47
    velocity: 100
    start_time: 0.5
    end_ti